![](https://www.dii.uchile.cl/wp-content/uploads/2021/06/Magi%CC%81ster-en-Ciencia-de-Datos.png)

# **Proyecto 1 - MDS7202 Laboratorio de Programación Científica para Ciencia de Datos 📚**

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

*Por favor, lean detalladamente las instrucciones de la tarea antes de empezar a escribir.*

### Equipo:

- Michelle Avendaño
- Claudia Navarro


### Link de repositorio de GitHub: `\<http://....\>`

Fecha límite de entrega 📆: 06 de Noviembre de 2023.

----

## Reglas

- **Grupos de 2 personas.**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Estrictamente prohibida la copia.
- Pueden usar cualquier material del curso que estimen conveniente.

<div style="text-align: center;">
    <img src="https://worldskateamerica.org/wp-content/uploads/2023/07/SANTIAGO-2023-1-768x153.jpg" alt="Descripción de la imagen">
</div>

En un Chile azotado por un profundo caos político-económico y el resurgimiento de programas de televisión de dudosa calidad, todas las miradas y esperanzas son depositadas en el éxito de un único evento: Santiago 2023. La nación necesitaba desesperadamente un respiro, y los Juegos de Santiago 2023 prometían ser una luz al final del túnel.

El Presidente de la República -conocido en las calles como Bombín-, consciente de la importancia de este evento para la revitalización del país, decide convocar a usted y su equipo en calidad de expertos en análisis de datos y estadísticas. Con gran solemnidad, el presidente les encomienda una importante y peligrosa: liderar un proyecto que permitiera caracterizar de forma automática y eficiente los datos generados por estos magnos juegos. Para esto, el presidente le destaca que la solución debe considerar los siguientes puntos:
- Caracterización automática de los datos
- La solución debe ser compatible con cualquier dataset
- Se les facilita el dataset *olimpiadas.parquet*, el cual recopila data de diferentes juegos olímpicos realizados en los últimos años

In [2]:
#importamos librería
import pandas as pd
import numpy as np
import os
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import re

In [28]:
data = pd.read_parquet('/content/olimpiadas.parquet')
data

,ID,Name,Sex,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,age-height-weight
0,1,A Dijiang,M,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None,24.0*180.0?80.0
1,2,A Lamusi,M,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None,23.0(170.0?60.0
2,3,Gunnar Nielsen Aaby,M,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None,24.0(nan?nan
3,4,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,34.0:nan?nan
4,5,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None,21.0(185.0?82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,None,29.0:179.0?89.0
271112,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",None,27.0:176.0?59.0
271113,135570,Piotr ya,M,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",None,27.0*176.0?59.0
271114,135571,Tomasz Ireneusz ya,M,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,None,30.0(185.0?96.0


## 1.1 Creación de `Profiler` Class (4.0 puntos)

Cree la clase `Profiler`. Como mínimo, esta debe tener las siguientes funcionalidades:

1. El método constructor, el cual debe recibir los datos a procesar en formato `Pandas DataFrame`. Además, este método debe generar una carpeta en su directorio de trabajo con el nombre `EDA_fecha`, donde `fecha` corresponda a la fecha de ejecución en formato `DD-MM-YYYY`.

2. El método `summarize`, el cual debe caracterizar las variables del Dataset. Como mínimo, se espera que su método pueda:
    - Implementar una funcionalidad para filtrar y aplicar este método a una o más variables de interés.
    - Reportar el tipo de variable
    - Reportar el número y/o porcentaje de valores únicos de la variable
    - Reportar el número y/o porcentaje de valores nulos
    - Si la variables es numérica:
        - Reportar el número y/o porcentaje de valores cero, negativos y outliers
        - Reportar estadística descriptiva como el valor mínimo, máximo, promedio y los percentiles 25, 50, 75 y 100
   - Levantar una alerta en caso de encontrar alguna anomalía fuera de lo común (el criterio debe ser ajustable por el usuario)
   - Guardar sus resultados en el directorio `EDA_fecha/summary.txt`. El archivo debe separar de forma clara y ordenada los resultados de cada punto.

3. El método `plot_vars`, el cual debe graficar la distribución e interraciones de las variables del Dataset. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/plots`
    - Implementar una funcionalidad para filtrar y aplicar este método a una o más variables de interés.
    - Para las variables numéricas:
        - Genere un gráfico de distribución de densidad
        - Grafique la correlación entre las variables
    - Para las variables categóricas:
        - Genere un histograma de las top N categorías (N debe ser un parámetro ajustable)
        - Grafique el coeficiente V de Cramer entre las variables
    - Guardar cada gráfico generado en la carpeta `EDA_fecha/plots` en formato `.pdf` y bajo el naming `variable.pdf`, donde `variable` es el nombre de la variable de interés
    
4. El método `clean_data`, el cual debe limpiar los datos para que luego puedan ser procesados. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/clean_data`
    - Implementar una funcionalidad para filtrar y aplicar este método a una o más variables de interés.
    - Drop de valores duplicados
    - Implementar como mínimo 2 técnicas para tratar los valores nulos, como:
        - Drop de valores nulos
        - Imputar valores nulos con alguna técnica de imputación
        - Funcionalidad para escoger entre una técnica y la otra.
    - Una de las columnas del dataframe presenta datos *no atómicos*. Separe dicha columna en las columnas que la compongan.
        - Hint: ¿Qué caracteres permiten separar una columna de otra?
        - Para las pruebas con el dataset nuevo, puede esperar que exista al menos una columna con este tipo de problema. Asuma que los separadores serán los mismos, aunque el número de columnas a separar puede ser distinto.
    - Deberían usar `FunctionTransformer`.
    - Guardar los datos procesados en formato `.csv` en el path `EDA_fecha/clean_data/data.csv`

5. El método `scale`, el cual debe preparar adecuadamente los datos para luego ser consumidos por algún tipo de algoritmo. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/scale`
    - Procesar de forma adecuada los datos numéricos y categóricos:
        - Su método debe recibir las técnicas de escalamiento como argumento de entrada (utilizar solo técnicas compatibles con el framework de `sklearn`)
        - Para los atributos numéricos, se transforme los datos con un escalador logarítmico y un `MinMaxScaler`
        - Asuma que no existen datos ordinales en su dataset
    - Guardar todo este procesamiento en un `ColumnTransformer`.
    - Guardar los datos limpios y transformados en formato `.csv` en el path `EDA_fecha/process/scaled_features.csv`

6. El método `make_clusters`, el cual debe generar clusters de los datos usando algún algoritmo de clusterización. Como mínimo, se espera que su método pueda:
    - Crear la carpeta `EDA_fecha/clusters`
    - Generar un estudio del codo donde señale la cantidad de clusters optimos para el desarrollo.
    - Su método debe recibir el algoritmo de clustering como argumento de entrada (utilizar solo algoritmos compatibles con el framework de `sklearn`).
    - No olvide pre procesar adecuadamente los datos antes de implementar la técnica de clustering.
    - En este punto es espera que generen un `Pipeline` de sklearn. Además, su método debería usar lo construido en los puntos 4 y 5.
    - Su método debe ser capaz de funcionar a partir de datos crudos (se descontará puntaje de lo contrario).
    - Una vez generado los clusters, proyecte los datos a 2 dimensiones usando su técnica de reducción de dimensionalidad favorita y grafique los resultados coloreando por cluster.
    - Guardar los datos con su respectivo cluster en formato `.csv` en el path `EDA_fecha/clusters/data_clusters.csv`. Guarde también los gráficos generados en el mismo path.

7. El método `detect_anomalies`, el cual debe detectar anomalías en los datos. Como mínimo, se espera que su método pueda:

    - Crear la carpeta `EDA_fecha/anomalies`
    - Implementar alguna técnica de detección de anomalías.
    - Al igual que el punto anterior, su método debe considerar los siguientes puntos:
        - No olvide pre procesar de forma adecuada los datos antes de implementar la técnica de detección de anomalía.
        - En este punto es espera que generen un `Pipeline` de sklearn. Además, su método debería usar lo construido en los puntos 4 y 5.
        - Su método debe ser capaz de funcionar a partir de datos crudos (se descontará puntaje de lo contrario).
        - Su método debe recibir el algoritmo como argumento de entrada
        - Una vez generado las etiquetas, proyecte los datos a 2 dimensiones y grafique los resultados coloreando por las etiquetas predichas por el detector de anomalías
    - Guardar los datos con su respectiva etiqueta en formato `.csv` en el path `EDA_fecha/anomalies/data_anomalies.csv`. Guarde también los gráficos generados en el mismo path.

8. El método `profile`, el cual debe ejecutar todos los métodos anteriores.

9. Crear el método `clearGarbage` para eliminar las carpetas/archivos creados/as por la clase `Profiler`.

Algunas consideraciones generales:
- Su clase será testeada con datos tabulares diferentes a los provistos. No desarrollen código *hardcodeado*: su clase debe ser capaz de funcionar para **cualquier** dataset.
- Aplique todo su conocimiento sobre buenas prácticas de programación: se evaluará que su código sea limpio y ordenado.
- Recuerden documentar cada una de las funcionalidades que implementen.
- Recuerden adjuntar sus `requirements.txt` junto a su entrega de proyecto. **El código que no se pueda ejecutar por imcompatibilidades de librerías no será corregido.**

In [95]:

class Profiler():

    def __init__(self, data: pd.DataFrame):
        """
        Inicializa la clase Profiler con el DataFrame proporcionado y crea una carpeta
        para almacenar los resultados del EDA con la fecha actual como referencia.

        Args:
            data (pd.DataFrame): El DataFrame a analizar.
        """
        self.data = data

        # Crear carpeta con el formato EDA_fecha
        self.folder_name = "EDA_" + datetime.datetime.now().strftime('%d-%m-%Y')
        if not os.path.exists(self.folder_name):
            os.makedirs(self.folder_name)

###SUMMARIZE###
    def summarize(self, cols=None, factor_outlier=1.5):
        """
        Resumen de estadísticas descriptivas para las columnas del DataFrame.
        Puede calcular y alertar sobre la presencia de valores atípicos usando el método del rango intercuartílico (IQR).

        Args:
            cols (list, optional): Columnas específicas a resumir. Si es None, resume todas las columnas.
            factor_outlier (float): Factor para identificar valores atípicos basado en el IQR.
        """
        if cols:
            df = self.data[cols]
        else:
            df = self.data

        summary = {}
        for col in df.columns:
            summary[col] = {}
            summary[col]['tipo'] = df[col].dtype
            summary[col]['valores_unicos'] = df[col].nunique()
            summary[col]['valores_nulos'] = df[col].isna().sum()


            if df[col].dtype in ['int64', 'float64']:
                summary[col]['valores_cero'] = (df[col] == 0).sum()
                summary[col]['valores_negativos'] = (df[col] < 0).sum()

                # Outliers
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                outliers = df[(df[col] < (Q1 - factor_outlier * IQR)) | (df[col] > (Q3 + factor_outlier * IQR))] #aqui influye el valor del factor
                if not outliers.empty:
                    print(f"¡Alerta! Se encontraron {len(outliers)} outliers en la columna '{col}'")
                summary[col]['outliers'] = len(outliers)

                # Estadísticas descriptivas
                summary[col]['min'] = df[col].min()
                summary[col]['25%'] = df[col].quantile(0.25)
                summary[col]['50%'] = df[col].quantile(0.50)
                summary[col]['75%'] = df[col].quantile(0.75)
                summary[col]['max'] = df[col].max()
        #Resumen
        with open(f'{self.folder_name}/summary.txt', 'w') as f:
            for k, v in summary.items():
                f.write(f"{k}:\n")
                for key, val in v.items():
                    f.write(f"  {key}: {val}\n")
                f.write("\n")
###PLOT_VARS###

    def _cramers_v(self, x, y):
        """ Función para calcular el coeficiente V de Cramer entre dos variables categóricas """
        confusion_matrix = pd.crosstab(x,y)
        chi2 = chi2_contingency(confusion_matrix)[0]
        n = confusion_matrix.sum().sum()
        phi2 = chi2/n
        r,k = confusion_matrix.shape
        phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
        rcorr = r-((r-1)**2)/(n-1)
        kcorr = k-((k-1)**2)/(n-1)
        return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

    def plot_vars(self, cols=None, top_n=10):
        """
        Método que genera y guarda visualizaciones para las distribuciones de las variables
        y sus correlaciones, tanto para variables numéricas como categóricas.

        Para las variables numéricas, crea gráficos de densidad de distribución y un mapa de calor
        de correlación. Para las variables categóricas, genera histogramas de las principales
        categorías y calcula el coeficiente V de Cramer entre pares de variables categóricas.

        Args:
            cols (list, optional): Lista de columnas para las que se generarán las visualizaciones.
                Si se omite, se utilizarán todas las columnas del DataFrame.
            top_n (int, optional): Número de principales categorías a visualizar en los histogramas
                para variables categóricas. Por defecto es 10.

        Notas:
            - Los gráficos generados se guardarán en el subdirectorio 'plots' dentro de la carpeta
              correspondiente al EDA con la fecha actual.
            - Los archivos de gráficos se guardan en formato PDF.
        """
        # Crear carpeta si no existe
        plots_folder = f'{self.folder_name}/plots'
        os.makedirs(plots_folder, exist_ok=True)

        if cols:
            df = self.data[cols]
        else:
            df = self.data

        # Graficar distribución y correlación para variables numéricas
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_cols:
            plt.figure(figsize=(10,6))
            sns.kdeplot(df[col], fill=True)
            plt.title(f'Densidad de distribución de {col}')
            plt.savefig(f'{plots_folder}/{col}.pdf')
            plt.close()

        if len(numeric_cols) > 1:
            plt.figure(figsize=(10,6))
            sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
            plt.title('Correlación de las variables numéricas')
            plt.savefig(f'{plots_folder}/correlation.pdf')
            plt.close()

      ###Categóricas###

        #Top N
        cat_cols = df.select_dtypes(include=['object']).columns
        for col in cat_cols:
            plt.figure(figsize=(10,6))
            df[col].value_counts().head(top_n).plot(kind='bar')
            plt.title(f'Top {top_n} categorías de {col}')
            plt.savefig(f'{plots_folder}/{col}_histogram.pdf')
            plt.close()

        # Calcular y graficar el coeficiente V de Cramer
        for i, col1 in enumerate(cat_cols):
            for j, col2 in enumerate(cat_cols):
                if i < j:  # para evitar repetir combinaciones
                    value = self._cramers_v(df[col1], df[col2])
                    plt.figure(figsize=(8, 5))
                    sns.barplot(x=[col2], y=[value])
                    plt.title(f"Cramer's V between {col1} and {col2}")
                    plt.ylabel("Cramer's V")
                    plt.savefig(f'{plots_folder}/{col1}_VS_{col2}_cramersV.pdf')
                    plt.close()

###CLEAN DATA###

    def _split_non_atomic(self, data, separators=[':', "(", "?", '*']):
            """
            Función para dividir las columnas que contienen múltiples valores en una sola celda.

            Args:
                data (pd.DataFrame): El DataFrame con posibles columnas no atómicas.
                separators (list): Lista de separadores utilizados para dividir las columnas.

            Returns:
                pd.DataFrame: DataFrame con las columnas divididas y valores convertidos a numéricos donde sea posible.
            """

            # Escapar los separadores para regex
            separators_regex = '|'.join(map(re.escape, separators))

            # Identificar columnas no numéricas con separadores
            non_atomic_columns = [
                column for column in data.columns if not pd.api.types.is_numeric_dtype(data[column]) and
                data[column].str.contains(separators_regex).any() and not data[column].str.contains("\)").any()
            ]

            # Procesar cada columna no atómica
            for column in non_atomic_columns:
                # Dividir la columna en múltiples columnas usando los separadores
                split_data = data[column].str.split(separators_regex, expand=True)

                # Asignar nuevos nombres a las columnas divididas
                split_columns = [f'{column}_{i}' for i in range(split_data.shape[1])]
                split_data.columns = split_columns

                # Convertir a numérico si es posible
                for split_column in split_columns:
                    split_data[split_column] = pd.to_numeric(split_data[split_column], errors='coerce')

                # Eliminar la columna original y unir las nuevas columnas al DataFrame
                data = data.drop(column, axis=1).join(split_data)

            # Reemplazar cadenas 'nan' con np.nan para uniformidad
            data.replace('nan', np.nan, inplace=True)
            return data


    def _cleaning_operations(self, data, method, nan_threshold=0.5, cols_to_clean=None):
        """
          Función para realizar operaciones de limpieza en el DataFrame, como eliminar columnas con
          un alto porcentaje de valores NaN o imputar valores faltantes.

          Args:
              data (pd.DataFrame): DataFrame a limpiar.
              method (str): Método de limpieza ('drop' o 'impute').
              nan_threshold (float): Umbral para la proporción de NaNs permitidos antes de eliminar una columna.
              cols_to_clean (list): Columnas específicas a limpiar. Si es None, limpia todo el DataFrame.

          Returns:
              pd.DataFrame: DataFrame limpio.
        """
        #Si es que hay columnas en específico que limpiar
        if cols_to_clean:
            df_to_clean = data[cols_to_clean].drop_duplicates()
        else:
            df_to_clean = data.drop_duplicates()
        #Eliminar columnas con muchos NAN
        threshold = len(df_to_clean) * nan_threshold
        df_to_clean = df_to_clean.dropna(thresh=threshold, axis=1)

        #Métodos para los NAN, de defecto es drop
        if method == "drop":
            df_to_clean.dropna(inplace=True)
        elif method == "impute":
            for col in df_to_clean.columns:
                if pd.api.types.is_numeric_dtype(df_to_clean[col]):
                    df_to_clean[col].fillna(df_to_clean[col].median(), inplace=True)
                else:
                    df_to_clean[col].fillna(df_to_clean[col].mode()[0], inplace=True) #Para las categóricas

        if cols_to_clean:
            # Reemplazar las columnas originales con las limpiadas en el DataFrame completo
            data.update(df_to_clean)
            return data
        else:
            return df_to_clean

    def clean_data(self, cols_to_clean=None, method="drop", nan_threshold=0.5):

        """
        Método  para limpiar el DataFrame. Divide primero las columnas no atómicas y luego realiza
        las operaciones de limpieza especificadas.

        Args:
            cols_to_clean (list): Columnas específicas a limpiar. Si es None, limpia todo el DataFrame.
            method (str): Método de limpieza a utilizar ('drop' o 'impute').
            nan_threshold (float): Umbral para la proporción de NaNs permitidos antes de eliminar una columna.

        Returns:
            pd.DataFrame: DataFrame limpio.
        """
        clean_data_folder = f'{self.folder_name}/clean_data'
        os.makedirs(clean_data_folder, exist_ok=True)

        # Aplicar _split_non_atomic a todo el DataFrame
        split_transformer = FunctionTransformer(self._split_non_atomic)
        df_split = split_transformer.transform(self.data.copy())

        # Aplicar _cleaning_operations solo a las columnas especificadas
        clean_transformer = FunctionTransformer(
            self._cleaning_operations,
            kw_args={'method': method, 'nan_threshold': nan_threshold, 'cols_to_clean': cols_to_clean}
        )
        df_clean = clean_transformer.transform(df_split)

        df_clean.to_csv(f'{clean_data_folder}/data.csv', index=False)
        return df_clean

###SCALE###




## 1.2 Caracterizar datos de Olimpiadas (2.0 puntos)

A partir de la clase que hemos desarrollado previamente, procederemos a realizar un análisis exhaustivo de los datos proporcionados en el enunciado. Este análisis se presentará en forma de un informe contenido en el mismo Jupyter Notebook y abordará los siguientes puntos:

1. Introducción
    - Se proporcionará una breve descripción del problema que estamos abordando y se explicará la metodología que se seguirá.

Elaborar una breve introducción con todo lo necesario para entender qué realizarán durante su proyecto. La idea es que describan de manera formal el proyecto con sus propias palabras y logren describir algunos aspectos básicos tanto del dataset como del análisis a realizar sobre los datos.

Por lo anterior, en esta sección ustedes deberán ser capaces de:

- Describir la tarea asociada al dataset.
- Describir brevemente los datos de entrada que les provee el problema.
- Plantear hipótesis de cómo podrían abordar el problema.

2. Análisis del EDA (Análisis Exploratorio de Datos)
    - Se discutirán las observaciones y conclusiones obtenidas acerca de los datos proporcionados. A lo largo de su respuesta, debe responder preguntas como:
        - ¿Como se comportan las variables numéricas? ¿y las categóricas?
        - ¿Existen valores nulos en el dataset? ¿En qué columnas? ¿Cuantos?
        - ¿Cuáles son las categorías y frecuencias de las variables categóricas?
        - ¿Existen datos duplicados en el conjunto?
        - ¿Existen relaciones o patrones visuales entre las variables?
        - ¿Existen anomalías notables o preocupantes en los datos?
3. Creación de Clusters y Anomalías
    - Se justificará la elección de los algoritmos a utilizar y sus hiperparámetros. En el caso de clustering, justifique además el número de clusters.
    
4. Análisis de Resultados
    - Se examinarán los resultados obtenidos a partir de los clústers y anomalías generadas. ¿Se logra una separación efectiva de los datos? Entregue una interpretación de lo que representa cada clúster y anomalía.
5. Conclusión
    - Se resumirán las principales conclusiones del análisis y se destacarán las implicaciones prácticas de los resultados obtenidos.